<a href="https://colab.research.google.com/github/NancWN/ScrumBusters/blob/main/KeywordSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

What can it do?:
it takes a predefined list of keywords which is divided between pump and no_pump words. it uses the list to search trough chat data. if a match is found, the group name, the word and label is written down in a seperate list.

Requirements:
1. upload a list with Keywords you want to search for
2. provide chat data from telegram groups in CSV format
3. adjust the path in the code

Lets go! :)

In [1]:
# settings
# feel free to change settings here
# only change other cells if you know what you are doing
keyword_set = 'brainstorm' # choose one of 'paper', 'brainstorm', 'mixed'
batch_number = 5
message_path = '/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/data/sample/messages_batch{}_old.csv'.format(batch_number) # put in the path of the message file

In [2]:
# run this cell to change working directory to its parent directory
%cd ..

/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram


In [3]:
# import libraries, load message data and keyword list
import csv
import pandas as pd 

#List of Keywords we want to search

lists = pd.read_csv('/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/Keyword Filter/references/{}_kw.csv'.format(keyword_set), sep=';', names=('pump','no_pump'))


pump_words=lists["pump"].dropna()
no_pump_words=lists["no_pump"].dropna()

#telegram chat export (csv-file)
searchlists=pd.read_csv(message_path, sep=',',na_values = ['no info', '.'])
searchlists.text= searchlists.text.astype('str')

             pump    no_pump
0            pump    no_pump
1            buy          go
2             bid      don't
3            sell    achieve
4          target       also
5         bittrex        may
6          signal        big
7      accumulate        ask
8             low    another
9            long       free
10  current price     profit
11          range   increase
12            day      trend
13      stop loss       news
14           good     bounce
15            buy    channel
16          start         pm
17          block     strong
18         volume  yesterday
19        accumul       done
20           coin     result
21             cp     period
22           stop        NaN
23           zone        NaN
24            min        NaN
25            max        NaN
26          entry        NaN


In [4]:
# create dataset of all chat names and chat ids
chat_names = searchlists.chat.unique()
chat_ids = range(len(chat_names))
chat_name_history = pd.DataFrame(data=[chat_names,chat_ids]).T
chat_name_history.columns = ['chat_name','chat_id']

# add the chat ids to the message dataframe
searchlists = searchlists.merge(
    chat_name_history,
    how='left',
    left_on = 'chat',
    right_on = 'chat_name'
    )
searchlists = searchlists.set_index('Unnamed: 0')
searchlists.index.name = 'row_id'
# drop the extended chat ids
searchlists.drop(['chat_name'],axis=1,inplace =True)
# simplify date column
searchlists.date = pd.to_datetime(searchlists.date).dt.tz_localize(None)
searchlists.reset_index(inplace = True)

In [5]:
searchlists

,row_id,text,date,id,chat,chat_id
0,286_Telegram_2022-03-10,anmeldecode: 94007. auf keinen fall diesen cod...,2022-03-10 14:57:55,777000,Telegram,0
1,278_Telegram_2022-02-26,anmeldecode: 11452. auf keinen fall diesen cod...,2022-02-26 12:10:08,777000,Telegram,0
2,276_Telegram_2022-02-09,login code: 39068. do not give this code to an...,2022-02-09 13:40:29,777000,Telegram,0
3,265_Telegram_2022-02-09,anmeldecode: 99866. auf keinen fall diesen cod...,2022-02-09 13:20:33,777000,Telegram,0
4,10580_𝘾𝙧𝙮𝙥𝙩𝙤 𝙂𝙪𝙧𝙪® 𝘽𝙞𝙣𝙖𝙣𝙘𝙚 𝙎𝙞𝙜𝙣𝙖𝙡𝙨_2022-03-10,#btc stucks in range between 35k to 45k range ...,2022-03-10 14:41:31,-1001093396548,𝘾𝙧𝙮𝙥𝙩𝙤 𝙂𝙪𝙧𝙪® 𝘽𝙞𝙣𝙖𝙣𝙘𝙚 𝙎𝙞𝙜𝙣𝙖𝙡𝙨,1
...,...,...,...,...,...,...
3426,7634_Crypto Insiders_2019-06-08,i am buying some powr here. looking for a next...,2019-06-08 07:40:00,-1001095533634,Crypto Insiders,24
3427,7633_Crypto Insiders_2019-06-08,buying steem at 477. target 10 - 20%,2019-06-08 05:53:07,-1001095533634,Crypto Insiders,24
3428,7632_Crypto Insiders_2019-06-07,whales are in $poe,2019-06-07 16:11:29,-1001095533634,Crypto Insiders,24
3429,20_Crypto Indicators_2019-11-14,📍click here for more telegram .groups! ➡️ @cry...,2019-11-14 15:54:24,-1001089139727,Crypto Indicators,25


In [6]:
# simple function to state if a substring was found in a string
# this is not necessary but it's easier to understand than alternative solutions
def key_found(key,msg):
    if key in str(msg):
        return True
    else:
        return False

In [7]:
# iterate over keyword list
# create new col per keyword to state if the keyword was found in the message
for keyword in pump_words:
    searchlists.loc[:,keyword] = [
        key_found(keyword, msg)
        for msg in searchlists.text
    ]
    
    '''searchlists.text = [
        msg.replace(keyword,'**'+keyword+'**')
        for msg in searchlists.text
    ]'''

# same process for no pump words
for keyword in no_pump_words:
    searchlists.loc[:,keyword] = [
        key_found(keyword, msg)
        for msg
        in searchlists.text
    ]
'''searchlists.text = [
        msg.replace(keyword,'##'+keyword+'##')
        for msg in searchlists.text
    ]'''

# get the number of keywords found per message
searchlists.loc[:,'n_pump_words'] = (searchlists[pump_words]).sum(axis=1)
searchlists.loc[:,'n_nopump_words'] = (searchlists[no_pump_words]).sum(axis=1)
searchlists.loc[:,'n_words'] = searchlists.n_pump_words + searchlists.n_nopump_words

searchlists.sort_values('n_words',ascending=False,inplace=True)
msg_sus_df = searchlists.loc[searchlists.n_words>0][['text','date','chat','n_words']]

In [8]:
msg_sus_df.to_csv('/Users/nancynyambura/Desktop/Digital Shaper Program/ScrumBusters/Frame 1/Telegram/chats/sus_msgs_{}_batch{}.csv'.format(keyword_set,batch_number))